In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.output {font-size:10pt;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
</style>
"""))

- class를 이용하여 출력하기 

In [38]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    
    def __str__(self):
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format(
            '*'*self.grade, self.name, self.phone, 
                self.email,self.age, self.etc)
    
    def to_dict(self):
        return {"name":self.name, "phone":self.phone,
               "email":self.email, "age":self.age,
               "grade":self.grade, "etc":self.etc}
    

def to_member(*row):
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

In [39]:
# 1. 입력
def insert_member_info():
    cursor = conn.cursor()
    
    name = input("이름: ")
    phone = input("전화번호: ")
    email = input("이메일: ")
    try:
        age = int(input('나이: '))
    except ValueError as e:
        print('유효하지 않은 나이 입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input('고객등급(1~5): '))
        if grade < 0:
            grade = 0
        elif grade > 5:
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input('기타: ')
    
    member = Member(name, phone, email, age, grade, etc)
    cursor.execute("""
        INSERT INTO MEMBER VALUES
        (:name, :phone, :email, :age, :grade, :etc)
    """, member.to_dict())
    
    conn.commit()
    conn.close()

In [ ]:
# 2. 전체 조회 
def print_members():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    
    members = cursor.fetchall()
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format
          ("GRADE","이름","전화","메일","나이","기타"))
    print('='*70)
    for member in members:
        print(to_member(*member))
    if len(members) == 0:
        print('저장된 회원이 없습니다.')
    print('='*70)
    
    cursor.close()

In [50]:
# 3.이름 찾기 
def search_member():
    cursor = conn.cursor()
    
    name = input('이름: ')
    corsur.execute("SELECT * FROM MEMBER WHERE NAME =:name", {'name':name})
    members = cursor.fetchall()
    
    for member in members:
        print(to_member(*member))
    if len(members) == 0:
        print('해당 이름의 회원이 조회되지 않습니다.')

In [42]:
# 4. 메일로 삭제하기
def delete_member():
    cursor = conn.cursor()
    
    email = input("이메일: ")
    cursor.execute("SELECT * FROM MEMBER WHERE EMAIL=:email",
                   {'email':email})
    members = cursor.fetchall()
    
    if len(members) != 0:
        cursor.execute("DELETE FROM MEMBER WHERE EMAIL=:emali",
                      {'email':email})
        conn.commit()
        print('요청하신 정보를 삭제하였습니다.')
    else:
        print('요청하신 정보가 존재하지 않습니다.')

In [43]:
# 5. csv파일로 내보내기 
def export_csv_member():
    filename = input('파일명(csv확장자 포함): ')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    colnames = [row[0] for row in cursor.description]
    cursor.close()
    
    import csv
    
    with open(filename, 'w', newline='', encoding='utf8') as file:
        w = csv.writer(file)
        w.writerow(colnames)
        w.writerow(members)

In [44]:
def main():
    while True:
        print("1:입력", "2:전체출력", "3:이름찾기", "4:메일삭제",
              "5:내보내기(CSV)", "0:종료", sep="|", end="")
        menu = int(input("메뉴 선택: "))
        
        if menu == 1:
            insert_member_info()
        elif menu == 2:
            print_members()
        elif menu == 3:
            search_member()
        elif menu == 4:
            delete_member()
        elif menu == 5:
            export_csv_member()
        elif menu == 0:
            conn.close()
            break;

In [ ]:
if __name__ == '__main__':
    import cx_Oracle
    global conn
    oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
    conn = cx_Oracle.connect('scott', 'tiger', dsn=oracle_dsn)
    main()

1:입력|2:전체출력|3:이름찾기|4:메일삭제|5:내보내기(CSV)|0:종료

- 클래스를 이용하지 않고 출력

In [8]:
# 1. 입력 
def insert_member_info():
    cursor = conn.cursor()
    name = input('이름: ')
    phone = input('전화번호: ')
    email = input('이메일: ')
    try:
        age = int(input('나이: '))
    except ValueError as e:
        print('유효하지 않은 값 입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade<0:
            grade = 0
        elif grade > 5:
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    cursor.execute("""
        INSERT INTO member VALUES
            (:name, :phone, :email, :age, :grade, :etc)
        """, {'name':name, 'phone':phone, 'email':email, 
              'age':age, 'grade':grade, 'etc':etc})
    conn.commit()
    cursor.close()

In [9]:
# 2. 전체 출력 
def print_members():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format("GRADE",
                                    "이름","전화","메일","나이","기타"))
    print('='*70)
    for member in members:
        print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format('*'*int(member[4]),member[0],
                                    member[1],member[2],member[3],member[5]))
    if len(members)==0:
        print("저장된 회원이 없습니다") 
    print('='*70)
    cursor.close()

In [11]:
# 3. 이름 찾기
def search_member():
    cursor = conn.cursor()
    name = input('검색할 이름을 입력하세요 ')
    cursor.execute("SELECT * FROM member WHERE NAME=:name", {'name':name})
    members = cursor.fetchall()
    for member in members:
        print("{}\t{}\t{}\t\t{}\t\t{}\t{}".format('*'*int(member[4]),member[0],
                                    member[1],member[2],member[3],member[5]))
    if len(members)==0:
        print("해당 이름의 회원이 조회되지 않습니다")

In [12]:
# 4. 메일 삭제 
def delete_member():
    cursor = conn.cursor()
    email = input('삭제할 회원의 이메일을 입력하세요 ')
    cursor.execute('SELECT * FROM MEMBER WHERE EMAIL=:email',{'email':email})
    members = cursor.fetchall()
    if len(members) != 0 :
        cursor.execute('DELETE FROM MEMBER WHERE EMAIL=:email',{'email':email})
        conn.commit()
        print('요청하신 메일을 삭제하였습니다')
    else :
        print('요청하신 메일이 데이터에 존재하지 않습니다')
    cursor.close()

In [13]:
# 5. 내보내기(csv)
def export_csv_member():
    file_name = input('파일명을 입력하세요(csv 확장자 포함)')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    colnames = [row[0] for row in cursor.description ]
    cursor.close()
    import csv
    with open(file_name, 'w', newline='', encoding='UTF8') as file:
        w = csv.writer(file)
        w.writerow(colnames)
        w.writerows(members)

In [7]:
def main():
    while True:
        print("1:입력","2:전체출력","3:이름찾기", "4:메일삭제","5:내보내기(CSV)", 
              "0:종료", sep="|", end="")
        menu = int(input("메뉴 선택 : "))
        if menu == 1:
            insert_member_info()
        elif menu ==2:
            print_members()
        elif menu == 3:
            search_member()
        elif menu == 4:
            delete_member()
        elif menu == 5:
            export_csv_member()
        elif menu == 0:
            conn.close()
            break;

In [ ]:
if __name__=='__main__':
    import cx_Oracle
    global conn
    oracle_dsn = cx_Oracle.makedsn(host='localhost', post=1521, sid='xe')
    conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
    main()